In [1]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.3 MB 9.0 MB/s 
     |████████████████████████████████| 684 kB 63.1 MB/s 
     |████████████████████████████████| 15.1 MB 22.8 MB/s 
     |████████████████████████████████| 348 kB 66.1 MB/s 
     |████████████████████████████████| 152 kB 66.0 MB/s 
     |████████████████████████████████| 470 kB 63.3 MB/s 
     |████████████████████████████████| 1.3 MB 55.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 425 kB 62.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81 kB 12.0 MB/s 
     |████████████████████████████████| 209 kB 72.2 MB/s 
     |████████████████████████████████| 78 kB 7.8 MB/s 
     |████████

In [2]:
import os   
def install_java():
  !apt update
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
install_java()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [91.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:14 http:

In [3]:
!wget http://faculty.washington.edu/browning/beagle/bref3.22Jul22.46e.jar
!wget http://faculty.washington.edu/browning/beagle/beagle.22Jul22.46e.jar

--2022-09-09 06:03:13--  http://faculty.washington.edu/browning/beagle/bref3.22Jul22.46e.jar
Resolving faculty.washington.edu (faculty.washington.edu)... 140.142.214.168, 128.208.60.35
Connecting to faculty.washington.edu (faculty.washington.edu)|140.142.214.168|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99396 (97K)
Saving to: ‘bref3.22Jul22.46e.jar’

bref3.22Jul22.46e.j 100%[===================>]  97.07K  --.-KB/s    in 0.02s   

2022-09-09 06:03:13 (3.90 MB/s) - ‘bref3.22Jul22.46e.jar’ saved [99396/99396]

--2022-09-09 06:03:13--  http://faculty.washington.edu/browning/beagle/beagle.22Jul22.46e.jar
Resolving faculty.washington.edu (faculty.washington.edu)... 140.142.214.168, 128.208.60.35
Connecting to faculty.washington.edu (faculty.washington.edu)|140.142.214.168|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 301565 (294K)
Saving to: ‘beagle.22Jul22.46e.jar’

beagle.22Jul22.46e. 100%[===================>] 294.50K  --.-KB/s    

In [49]:
from fuc import pyvcf, common
import pandas as pd
import numpy as np
import random
from tqdm.notebook import tqdm

In [64]:
#sample = sample.applymap(lambda x: x.replace('|', '/') if type(x) == str else x)[sample.columns[:59]]

нужно три файла: 

1.   семпл из референсной панели
2.   семпл с пропущенными значениями
3.   семпл с заполненными значениями

Последовательность действий:

1.   Семплируем 
2.   Восстанавливаем
3.   Берем исходные значения, которые были на месте пропусков 
4.   Берем восстановленные значения
5.   Сравниваем из между собой 





In [112]:
# Этап 1 - загружаем данные референсной панели
chr = pyvcf.VcfFrame.from_file('/content/ref_data/melon_impute_chr1.vcf')
chr_data = chr.df
chr_data.columns = list(chr_data.columns[:9]) + [f'sample_{i}' for i in range(2084)]
chr_data.head(3)

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,sample_0,...,sample_2074,sample_2075,sample_2076,sample_2077,sample_2078,sample_2079,sample_2080,sample_2081,sample_2082,sample_2083
0,1,4372,S1_4372,C,G,.,PASS,.,GT,0|0,...,0|0,1|1,0|0,0|1,0|0,0|0,0|0,0|1,0|0,0|0
1,1,20201,S1_20201,T,G,.,PASS,.,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
2,1,27184,S1_27184,C,A,.,PASS,.,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0


In [184]:
#этап 2 - функция для семплирования 
def replaceRandom(arr, p):
    temp = np.asarray(arr)   # Cast to numpy array
    shape = temp.shape       # Store original shape
    temp = temp.flatten()    # Flatten to 1D
    
    inds = np.random.choice(temp.size, size=int(p*shape[0]*shape[1]))  # Get random indices
    temp[inds] = '.|.'          # Fill with something
    temp = temp.reshape(shape)                     # Restore original shape
    return temp

def make_sample(chr_data, n, p1, p2):
    s = random.sample(range(0, 2083), 25) 
    sample = chr_data.sample(n = n, replace=False).sort_values('POS').reset_index(drop=True)
    sample = sample[ list(sample.columns[:9] )+ [f'sample_{i}' for i in s]]
    sample_with_nas = replaceRandom(np.array(sample[[f'sample_{i}' for i in s]]), random.uniform(p1, p2))
    sample_init = sample.copy()
    sample[[f'sample_{i}' for i in s]] = sample_with_nas
    return sample, sample_init

In [167]:
#a, b = make_sample(chr_data, 100, 0, 0.1)

In [185]:
# готовим данные для обучения
for i in tqdm(range(10)):
  for l in [50, 100, 250, 500]:
    for j, k in [(0,0.1), (0.1, 0.3), (0.3, 1)]: 
      sample, sample_init = make_sample(chr_data, l, j, k)
      sample.to_csv(f'sample{i}{l}{j*10}{k*10}.csv', index = False)
      sample_init.to_csv(f'sample_init{i}{l}{j*10}{k*10}.csv', index = False)

  0%|          | 0/10 [00:00<?, ?it/s]

In [186]:
sample = pd.read_csv(f'sample{i}{l}{j*10}{k*10}.csv')
sample_init = pd.read_csv(f'sample_init{i}{l}{j*10}{k*10}.csv')

In [187]:
sample_init.head(3)

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,sample_1458,...,sample_1672,sample_810,sample_1365,sample_1652,sample_1447,sample_2074,sample_274,sample_1771,sample_233,sample_1593
0,1,53942,S1_53942,C,T,.,PASS,.,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
1,1,82128,S1_82128,T,G,.,PASS,.,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
2,1,123609,S1_123609,A,G,.,PASS,.,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,1|1,0|0,1|1


In [188]:
sample.head(3)

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,sample_1458,...,sample_1672,sample_810,sample_1365,sample_1652,sample_1447,sample_2074,sample_274,sample_1771,sample_233,sample_1593
0,1,53942,S1_53942,C,T,.,PASS,.,GT,0|0,...,.|.,.|.,.|.,0|0,0|0,0|0,0|0,0|0,.|.,.|.
1,1,82128,S1_82128,T,G,.,PASS,.,GT,0|0,...,0|0,.|.,.|.,.|.,.|.,0|0,0|0,0|0,0|0,.|.
2,1,123609,S1_123609,A,G,.,PASS,.,GT,0|0,...,.|.,0|0,0|0,0|0,0|0,.|.,0|0,1|1,0|0,1|1


In [189]:
# все файла сохраним в vcf
for i in tqdm(range(10)):
  for l in [50, 100, 250, 500]:
    for j, k in [(0,0.1), (0.1, 0.3), (0.3, 1)]: 
      sample = pd.read_csv(f'sample{i}{l}{j*10}{k*10}.csv')
      sample = pyvcf.VcfFrame(['##fileformat=VCFv4.3'], sample)
      sample.to_file(f'/content/sampled_data/sample{i}{l}{j*10}{k*10}.vcf')

  0%|          | 0/10 [00:00<?, ?it/s]

In [238]:
# Этап три - функция для восстановления пропущенных значений
def impute_values(sample_name, df_name, params=''):
  os.system(f'java -jar beagle.22Jul22.46e.jar ref=/content/melon_impute_Chr1.vcf.gz  gt={sample_name} out=out {params}')
  os.system('gunzip out.vcf.gz')
  sample = pd.read_csv(df_name)

  imputed_sample = pyvcf.VcfFrame.from_file('/content/out.vcf')
  imputed_sample = imputed_sample.df

  imputed_sample = imputed_sample[imputed_sample.POS.isin(sample.POS)][sample.columns].applymap(lambda x: x.split(':')[0] if type(x) == str else x)

  os.system('rm out.vcf')

  return imputed_sample

In [239]:
%%time
imp_sampel = impute_values('/content/sampled_data/sample35003.010.vcf', '/content/sample35003.010.csv', params='ne=1000')

CPU times: user 82.4 ms, sys: 58.9 ms, total: 141 ms
Wall time: 7.58 s


In [229]:
# Проверка качества работы алгоритма

def get_accuracy(sample, sample_init, imp_sample):
  mask = np.array(sample[sample.columns[9:]]) == '.|.'
  init = np.array(sample_init[sample.columns[9:]])[mask]
  imputed = np.array(imp_sample[sample.columns[9:]])[mask]
  return (init == imputed).sum()/len(init)

In [230]:
imp_sample = impute_values('/content/sampled_data/sample35003.010.vcf', '/content/sample35003.010.csv', params='')
sample = pd.read_csv('/content/sample35003.010.csv')
sample_init = pd.read_csv('/content/sample_init35003.010.csv')

get_accuracy(sample, sample_init, imp_sample)

0.9834564728557661

In [ ]:
#Проверим качетсво бейзлайна

accuracies = {'0_0_1': [],
              '0_1_0_3': [],
              '0_3_1': []}

for i in tqdm(range(10)):
  for l in tqdm([50, 100, 250, 500]):
    for j, k in tqdm([(0,0.1), (0.1, 0.3), (0.3, 1)]): 
      imp_sample = impute_values(f'/content/sampled_data/sample{i}{l}{j*10}{k*10}.vcf', f'sample{i}{l}{j*10}{k*10}.csv', params='')
      sample = pd.read_csv(f'sample{i}{l}{j*10}{k*10}.csv')
      sample_init = pd.read_csv(f'sample_init{i}{l}{j*10}{k*10}.csv')
      if j == 0:
        accuracies['0_0_1'].append(get_accuracy(sample, sample_init, imp_sample))
      elif j == 0.1:
        accuracies['0_1_0_3'].append(get_accuracy(sample, sample_init, imp_sample))
      else:
        accuracies['0_3_1'].append(get_accuracy(sample, sample_init, imp_sample))



In [233]:
print(np.mean(accuracies['0_0_1']))
print(np.mean(accuracies['0_1_0_3']))
print(np.mean(accuracies['0_3_1']))

0.962687007894683
0.9627373919752777
0.9396656993021149


In [249]:
# Подбор гиперпараметров
import optuna

def objective(trial):

    param = {
        "imp-states": trial.suggest_int("imp-states", 1200, 3000, step=100),
        "imp-segment": trial.suggest_int("imp-segment", 3, 10, step=1),
        "imp-step": trial.suggest_float("imp-step", 0.01, 0.3),
        "imp-nsteps": trial.suggest_int("imp-nsteps", 5, 12, step=1),
        "ne": trial.suggest_int("ne", 10000, 1000000, step=10000),
        "overlap": trial.suggest_int("overlap", 1, 4, step=1),
        "window": trial.suggest_int("window", 30, 70, step=5)
    }

    params = f"imp-states={param['imp-states']} imp-segment={param['imp-segment']} imp-step={param['imp-step']} imp-nsteps={param['imp-nsteps']} ne={param['ne']} overlap={param['overlap']} window={param['window']}"

    accuracy = []

    for i in range(10):
      for l in [50, 100, 250, 500]:
        for j, k in [(0.3, 1)]: 

          imp_sample = impute_values(f'/content/sampled_data/sample{i}{l}{j*10}{k*10}.vcf', f'sample{i}{l}{j*10}{k*10}.csv', params=params)
          sample = pd.read_csv(f'sample{i}{l}{j*10}{k*10}.csv')
          sample_init = pd.read_csv(f'sample_init{i}{l}{j*10}{k*10}.csv')
          accuracy.append(get_accuracy(sample, sample_init, imp_sample))


    return np.mean(accuracy)




In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, timeout=7200)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-09-09 12:56:20,434] A new study created in memory with name: no-name-f50f7a2a-05f0-45dd-a076-5957b8a386cf


In [ ]:
# Проверка качества подобранных параметров

accuracies = {'0_0_1': [],
              '0_1_0_3': [],
              '0_3_1': []}
              
param = study.best_trial
params = f"imp-states={param['imp-states']} imp-segment={param['imp-segment']} imp-step={param['imp-step']} imp-nsteps={param['imp-nsteps']} ne={param['ne']} overlap={param['overlap']} window={param['window']}"


for i in tqdm(range(10)):
  for l in tqdm([50, 100, 250, 500]):
    for j, k in tqdm([(0,0.1), (0.1, 0.3), (0.3, 1)]): 
      imp_sample = impute_values(f'/content/sampled_data/sample{i}{l}{j*10}{k*10}.vcf', f'sample{i}{l}{j*10}{k*10}.csv', params=params)
      sample = pd.read_csv(f'sample{i}{l}{j*10}{k*10}.csv')
      sample_init = pd.read_csv(f'sample_init{i}{l}{j*10}{k*10}.csv')
      if j == 0:
        accuracies['0_0_1'].append(get_accuracy(sample, sample_init, imp_sample))
      elif j == 0.1:
        accuracies['0_1_0_3'].append(get_accuracy(sample, sample_init, imp_sample))
      else:
        accuracies['0_3_1'].append(get_accuracy(sample, sample_init, imp_sample))

In [ ]:
print(np.mean(accuracies['0_0_1']))
print(np.mean(accuracies['0_1_0_3']))
print(np.mean(accuracies['0_3_1']))